In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
import time
from sklearn.externals import joblib
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier
from catboost import CatBoostClassifier

/home/veruska/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
city = 'Recife'

data_path = '../../data/output/' + city + '/normalized_data_X.csv'
y_path = '../../data/output/' + city + '/y.csv'

data = pd.read_csv(data_path)
y = pd.read_csv(y_path)
print(data.shape)
print(y.shape)
data.head()

(1432633, 420)
(1432633, 1)


,route,tripNum,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,...,alertTypeSB_ACCIDENT,alertTypeSB_CHIT_CHAT,alertTypeSB_HAZARD,alertTypeSB_JAM,alertTypeSB_NORMAL,alertTypeSB_POLICE,alertTypeSB_ROAD_CLOSED,jamBlockTypeSB_-,jamBlockTypeSB_NORMAL,jamBlockTypeSB_ROAD_CLOSED_EVENT
0,0.118068,0.066667,0.085019,0.264600,0.496150,0.057584,0.0,0.000000,0.266155,0.496047,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.118068,0.066667,0.085026,0.270218,0.501802,0.064231,0.0,0.000148,0.271812,0.501642,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.118068,0.066667,0.085033,0.273653,0.505296,0.068301,0.0,0.000000,0.275187,0.505192,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.118068,0.066667,0.085039,0.277433,0.509404,0.072900,0.0,0.000149,0.280622,0.510132,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.118068,0.066667,0.085053,0.280050,0.513166,0.077983,0.0,0.000155,0.281878,0.513178,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
data.sort_values(by=['busCode', "busCodeSB", "DAY(gps_datetime)", "GPShour"], inplace=True)
data

,route,tripNum,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,...,alertTypeSB_ACCIDENT,alertTypeSB_CHIT_CHAT,alertTypeSB_HAZARD,alertTypeSB_JAM,alertTypeSB_NORMAL,alertTypeSB_POLICE,alertTypeSB_ROAD_CLOSED,jamBlockTypeSB_-,jamBlockTypeSB_NORMAL,jamBlockTypeSB_ROAD_CLOSED_EVENT
0,0.118068,0.066667,0.085019,0.264600,0.496150,0.057584,0.0,0.000000,0.266155,0.496047,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.118068,0.066667,0.085026,0.270218,0.501802,0.064231,0.0,0.000148,0.271812,0.501642,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.118068,0.066667,0.085033,0.273653,0.505296,0.068301,0.0,0.000000,0.275187,0.505192,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.118068,0.066667,0.085039,0.277433,0.509404,0.072900,0.0,0.000149,0.280622,0.510132,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.118068,0.066667,0.085053,0.280050,0.513166,0.077983,0.0,0.000155,0.281878,0.513178,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432628,0.243292,0.033333,0.264290,0.358156,0.908889,0.225439,1.0,0.000000,0.359504,0.908700,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1432629,0.243292,0.033333,0.264648,0.419832,0.936254,0.300294,1.0,0.972305,0.422069,0.933279,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1432630,0.243292,0.033333,0.264655,0.421938,0.930378,0.304917,1.0,0.000000,0.423144,0.930185,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1432631,0.243292,0.033333,0.264695,0.427121,0.915945,0.316300,1.0,0.000000,0.428316,0.915756,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [3]:
y.rename(columns={"busBunching": "busBunching0"}, inplace=True)

y['busBunching1'] = 0
y['busBunching2'] = 0
y['busBunching3'] = 0
y['busBunching4'] = 0
y['busBunching5'] = 0

y.head()

,busBunching0,busBunching1,busBunching2,busBunching3,busBunching4,busBunching5
0,0.0,0,0,0,0,0
1,0.0,0,0,0,0,0
2,0.0,0,0,0,0,0
3,0.0,0,0,0,0,0
4,0.0,0,0,0,0,0


In [7]:
start = time.time()

for i in range(data.shape[0]-5):
    
    current_bus_code = data['busCode'][i]
    current_bus_codeSB = data['busCodeSB'][i]
    
    if ((data['busCode'][i+1] == current_bus_code) & (data['busCodeSB'][i+1] == current_bus_codeSB)):
        y['busBunching1'][i] = y['busBunching0'][i+1]
    #else:
        #what to put when does not exists?
        
    if ((data['busCode'][i+2] == current_bus_code) & (data['busCodeSB'][i+2] == current_bus_codeSB)):
        y['busBunching2'][i] = y['busBunching0'][i+2]
    #else:
        #what to put when does not exists?
        
    if ((data['busCode'][i+3] == current_bus_code) & (data['busCodeSB'][i+3] == current_bus_codeSB)):
        y['busBunching3'][i] = y['busBunching0'][i+3]
    #else:
        #what to put when does not exists?
        
    if ((data['busCode'][i+4] == current_bus_code) & (data['busCodeSB'][i+4] == current_bus_codeSB)):
        y['busBunching4'][i] = y['busBunching0'][i+4]
    #else:
        #what to put when does not exists?
    
    if ((data['busCode'][i+5] == current_bus_code) & (data['busCodeSB'][i+5] == current_bus_codeSB)):
        y['busBunching5'][i] = y['busBunching0'][i+5]
    #else:
        #what to put when does not exists?

end = time.time()
print("Execution time: " + str((end - start)/60) + " min")

/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/lib/python3/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/lib/python3/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be

Execution time: 18.931247492631275 min


In [17]:
y[y.busBunching5 == "-"]

,busBunching0,busBunching1,busBunching2,busBunching3,busBunching4,busBunching5
324,0.0,0,0,0,0,-
325,0.0,0,0,0,-,-
326,0.0,0,0,-,-,-
327,0.0,0,-,-,-,-
328,0.0,-,-,-,-,-
...,...,...,...,...,...,...
1432628,0.0,-,-,-,-,-
1432629,0.0,-,-,-,-,-
1432630,0.0,-,-,-,-,-
1432631,0.0,-,-,-,-,-


In [5]:
y['busBunching1'][y['busBunching1'] == 0]

0          0
1          0
2          0
3          0
4          0
          ..
1432628    0
1432629    0
1432630    0
1432631    0
1432632    0
Name: busBunching1, Length: 1432630, dtype: int64

In [6]:
data[['busCode','busCodeSB','stopPointId']].head(10)

,busCode,busCodeSB,stopPointId
0,0.0,0.000014,0.337464
1,0.0,0.000014,0.337602
2,0.0,0.000014,0.337739
3,0.0,0.000014,0.337877
4,0.0,0.000014,0.297398
5,0.0,0.000014,0.260774
6,0.0,0.000014,0.260498
7,0.0,0.000014,0.260223
8,0.0,0.000014,0.259948
9,0.0,0.000014,0.259672


In [8]:
# Making training and test data: 80% Training, 20% Test
random.seed(15) #to get always the same set
train_X, test_X, train_Y, test_Y = train_test_split(data, y, test_size=0.20, random_state=7)

del data
del y

### Ensemble Multi-Step (5-step)

In [ ]:
# preparar a base, adicionar outros 5ys (5 predições a frente)
# retreinar os modelos
# executar ensemble
# calcular acurácia

In [6]:
random.seed(42)

best_estimator_rf = 100
best_split_rf = 5
best_max_feat_rf = 0.8

rf = RandomForestClassifier(n_estimators=best_estimator_rf, min_samples_split=best_split_rf,
                           max_features=best_max_feat_rf, n_jobs=-1)

xgb = XGBClassifier(
 learning_rate =0.1,
 n_estimators=120, 
 max_depth=50,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=1,
 seed=23)


catboost = CatBoostClassifier(
    custom_loss=['Accuracy', 'Precision', 'Recall', 'F1'],
    iterations=10000,
    learning_rate=0.9,
    random_seed=42,
    depth=8,
    l2_leaf_reg=5,
    logging_level='Silent'
)

In [5]:
output_path = '../../data/output/' + city + '/'
cols=['n0', 'n1', 'n2', 'n3', 'n4', 'n5']

In [ ]:
start = time.time()

multi_target_forest_rf = MultiOutputClassifier(rf)
multi_target_forest_rf.fit(train_X, train_Y)

end = time.time()
print("Execution time: " + str((end - start)/60) + " min")

start_p = time.time()
rf_pred = multi_target_forest_rf.predict(test_X)
end_p = time.time()
print("Prediction execution time: " + str((end_p - start_p)/60) + " min")
del multi_target_forest_rf
rf_pred_pd = pd.DataFrame(rf_pred, columns=cols)
rf_pred_pd.to_csv(output_path + "rf_pred_multi.csv", index=False)

In [16]:
rf_pred = pd.read_csv(output_path + "rf_pred_multi.csv").values
rf_pred

array([[0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [17]:
xgb_pred = pd.read_csv(output_path + "xgb_pred_multi.csv").values
cat_pred = pd.read_csv(output_path + "cat_pred_multi.csv").values

In [ ]:
start = time.time()

multi_target_forest_xgb = MultiOutputClassifier(xgb)
multi_target_forest_xgb.fit(train_X, train_Y)

end = time.time()
print("Execution time: " + str((end - start)/60) + " min")

start_p = time.time()
xgb_pred = multi_target_forest_xgb.predict(test_X)
end_p = time.time()
print("Prediction execution time: " + str((end_p - start_p)/60) + " min")
del multi_target_forest_xgb
xgb_pred_pd = pd.DataFrame(xgb_pred, columns=cols)
xgb_pred_pd.to_csv(output_path + "xgb_pred_multi.csv", index=False)

In [ ]:
start = time.time()

multi_target_forest_catboost = MultiOutputClassifier(catboost)
multi_target_forest_catboost.fit(train_X, train_Y)

end = time.time()
print("Execution time: " + str((end - start)/60) + " min")

start_p = time.time()
cat_pred = multi_target_forest_catboost.predict(test_X)
end_p = time.time()
print("Prediction execution time: " + str((end_p - start_p)/60) + " min")
del multi_target_forest_catboost
cat_pred_pd = pd.DataFrame(cat_pred, columns=cols)
cat_pred_pd.to_csv(output_path + "cat_pred_multi.csv", index=False)

In [ ]:
test_Y.head(10)

In [18]:
n_steps = 5 + 1 #+1 for
predictions = []

for i in range(0, len(rf_pred)):
    pred_steps = []
    
    for n in range(0, n_steps):
        temp_preds = []
        temp_preds.append(rf_pred[i, n])
        temp_preds.append(cat_pred[i, n])
        temp_preds.append(xgb_pred[i, n])

        pred_steps.append(
            max(temp_preds,key=temp_preds.count)
        )
        
    predictions.append(pred_steps)
    
final_predictions = np.array(predictions)

In [19]:
final_predictions

array([[0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [34]:
test_Y[test_Y.busBunching2 == 5.0].head(50)

,busBunching0,busBunching1,busBunching2,busBunching3,busBunching4,busBunching5


In [20]:
for n in range(0, n_steps):
    print("Step: " + str(n))
    accuracy = accuracy_score(test_Y['busBunching' + str(n)], final_predictions[:,n])
    precision = precision_score(test_Y['busBunching' + str(n)], final_predictions[:,n])
    recall = recall_score(test_Y['busBunching' + str(n)], final_predictions[:,n])
    f_measure = f1_score(test_Y['busBunching' + str(n)], final_predictions[:,n])
    
    print('Accuracy: ' + str(accuracy))
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall))
    print('F-measure: ' + str(f_measure))

Step: 0
Accuracy: 0.9521301657435425
Precision: 0.905814371061996
Recall: 0.719236850973378
F-measure: 0.8018148190960582
Step: 1
Accuracy: 0.9443577743109725
Precision: 0.8719975980784628
Recall: 0.6834618904431952
F-measure: 0.7663036308468067
Step: 2
Accuracy: 0.9416250475522376
Precision: 0.8670875218718908
Recall: 0.6628983606557377
F-measure: 0.7513675823522417
Step: 3
Accuracy: 0.9413388616081556
Precision: 0.868606205586435
Recall: 0.6538237474843767
F-measure: 0.746064360175253
Step: 4
Accuracy: 0.941656458204637
Precision: 0.873470413033337
Recall: 0.6485399126079079
F-measure: 0.7443844707105614
Step: 5
Accuracy: 0.9402883497890251
Precision: 0.8706751517811425
Recall: 0.6368816715267931
F-measure: 0.7356499436040851
